# Лабораторная работа №1 "Градиентный спуск и его модификации"

## Задание

1. Градиентный спуск и его модификации
   - Выбрать [тестовые функции оптимизации](https://ru.wikipedia.org/wiki/Тестовые_функции_для_оптимизации) (2 шт)
   - Запрограммировать собственную реализацию классического градиентного спуска
   - Запрограммировать пайлайн тестирования алгоритма оптимизации
     - Визуализации функции и точки оптимума
     - Вычисление погрешности найденного решения в сравнение с аналитическим для нескольких запусков
     - Визуализации точки найденного решения (можно добавить анимацию на плюс балл)
   - Запрограммировать метод вычисления градиента
     - Передача функции градиента от пользователя
     - Символьное вычисление градиента (например с помощью [sympy](https://www.sympy.org/en/index.html)) (на доп балл)
     - Численная аппроксимация градиента (на доп балл)
   - Запрограммировать одну моментную модификацию и протестировать ее
   - Запрограммировать одну адаптивную модификацию и протестировать ее
   - Запрограммировать метод эфолюции темпа обучения и/или метод выбора начального приближения и протестировать их
   - `Will be unclocked afetr Lecture №5`

## Решение

В качестве тестовых функций возьмем: 
- **функция Розенброка** 

1. $ f(x) = \sum _ {i=1} ^ {n-1} [100(x_{i+1}-x_{i}^{2})^2+(x_i-1)^2]$

![](https://upload.wikimedia.org/wikipedia/commons/thumb/7/7e/Rosenbrock%27s_function_in_3D.pdf/page1-640px-Rosenbrock%27s_function_in_3D.pdf.jpg?uselang=ru)

2. Минимум: $ f(1_0, 1_1, ..., 1_n)=0 $

3. Метод поиска: $-\infty \leq x_i \leq \infty, 1 \leq i \leq n$

- **функция Била**

1. $ f(x,y) = (1.5-x+xy)^2+(2.25-x+xy^2)^2+(2.625-x+xy^3)^2 $

![](https://upload.wikimedia.org/wikipedia/commons/thumb/d/de/Beale%27s_function.pdf/page1-640px-Beale%27s_function.pdf.jpg?uselang=ru)

2. Минимум: $ f(3, 0.5) = 0 $

3. Метод поиска: $-4.5 \leq x,y \leq 4.5$

In [9]:
import numpy as np
import sympy as sp
import plotly.graph_objects as go

Напишем вычисление выбранных тестовых функций. Для функции Розенброка реализуем вариант функции от двух переменных.

In [23]:
def rosenbrock(point):
    x, y = point[0], point[1]
    return 100 * (y - x**2)**2 + (x - 1)**2

def beale(point):
    x, y = point[0], point[1]
    return ((1.5 - x + x * y)**2 +
            (2.25 - x + x * y**2)**2 +
            (2.625 - x + x * y**3)**2)

Реализуем классический алгоритм градиентного спуска. На данном этапе будем подавать при вызове функцию для подсчета градиента. Позже, после выполнения задания по численной аппроксимации, сосздадим еще одну имплементацию, которая уже будет использовать приблежнные значения градиента.

In [11]:
class GradientDescent:
    def __init__(self, learning_rate=0.01, max_iterations=1000, tolerance=1e-6):
        """
        Конструктор класса GradientDescent.

        :param learning_rate: Скорость обучения (шаг градиентного спуска).
        :param max_iterations: Максимальное количество итераций.
        :param tolerance: Критерий остановки, если изменения функции невелики.
        """
        self.learning_rate = learning_rate
        self.max_iterations = max_iterations
        self.tolerance = tolerance
        self.path = []  # Для сохранения траектории точек

    def minimize(self, func, grad_func, initial_point):
        """
        Метод для минимизации функции с использованием градиентного спуска.

        :param func: Целевая функция, которую нужно минимизировать.
        :param grad_func: Градиент целевой функции.
        :param initial_point: Начальная точка для поиска минимума.
        :return: Кортеж (минимальное значение функции, точка минимума, путь градиента).
        """
        point = np.array(initial_point)
        self.path = [point.copy()]  # Сохраняем начальную точку

        for i in range(self.max_iterations):
            gradient = np.array(grad_func(point))
            new_point = point - self.learning_rate * gradient

            # Проверка критерия остановки
            if np.linalg.norm(new_point - point) < self.tolerance:
                print(f"Сошлось за {i+1} итераций.")
                self.path.append(new_point.copy())
                return func(new_point), new_point, self.path

            point = new_point
            self.path.append(point.copy())  # Сохраняем текущую точку

        print("Достигнуто максимальное количество итераций.")
        return func(point), point, self.path

Напишем пайплайн визуализации и подсчета ошибки

In [12]:
def visualize_optimization_plotly(function, x_min, y_min, optimization_paths):
    """
    Визуализирует функцию, точку минимума и пути оптимизации в 3D с использованием Plotly.

    :param function: Целевая функция.
    :param x_min: Реальная x-координата минимума.
    :param y_min: Реальная y-координата минимума.
    :param optimization_paths: Список путей градиентного спуска. Каждый путь — это массив точек [(x1, y1), ..., (xn, yn)].
    """
    # Создание сетки для 3D-графика
    x = np.linspace(-2, 2, 100)
    y = np.linspace(-1, 3, 100)
    X, Y = np.meshgrid(x, y)
    Z = function(X, Y)

    # 3D поверхность функции
    surface = go.Surface(z=Z, x=X, y=Y, colorscale='Viridis', opacity=0.8)

    # Точка минимума
    minimum_point = go.Scatter3d(
        x=[x_min],
        y=[y_min],
        z=[function(x_min, y_min)],
        mode='markers',
        marker=dict(size=6, color='red'),
        name='Точка минимума'
    )

    # Пути оптимизации
    paths = []
    for idx, path in enumerate(optimization_paths):
        xs, ys = zip(*path)
        zs = [function(x, y) for x, y in path]
        paths.append(
            go.Scatter3d(
                x=xs,
                y=ys,
                z=zs,
                mode='lines+markers',
                name=f'Путь {idx + 1}'
            )
        )

    # Настройка графика
    fig = go.Figure(data=[surface, minimum_point] + paths)
    fig.update_layout(
        scene=dict(
            xaxis_title='X',
            yaxis_title='Y',
            zaxis_title='f(X, Y)',
        ),
        title="3D-визуализация функции и пути оптимизации",
        margin=dict(l=0, r=0, b=0, t=40)
    )
    fig.show()


def calculate_errors(function, x_min, y_min, optimization_paths):
    """
    Вычисляет погрешности по модулю для каждого прогона и среднюю погрешность.
    """
    errors = []
    for path in optimization_paths:
        x_opt, y_opt = path[-1]  # Последняя точка пути — точка завершения
        error = np.sqrt((x_opt - x_min)**2 + (y_opt - y_min)**2)
        errors.append(error)

    mean_error = np.mean(errors)

    print("Погрешности по каждому прогону:")
    for i, error in enumerate(errors, 1):
        print(f"Прогон {i}: Погрешность = {error:.6f}")

    print(f"\nСредняя погрешность: {mean_error:.6f}")

    return errors, mean_error


def pipeline(function, x_min, y_min, optimization_paths):
    # Визуализация
    visualize_optimization_plotly(function, x_min, x_min, optimization_paths)

    # Вычисление погрешностей
    calculate_errors(function, x_min, x_min, optimization_paths)

Поскольку метод градиентного спуска реализован пока что через подсчет по готовой функции градиента, воспользуемся библиотекой sympy для нахождения функции градиента.

In [13]:
def compute_gradient(func):
    # Запрос функции от пользователя
    expr_str = func
    
    # Парсим введенную строку как выражение sympy
    expr = sp.sympify(expr_str)
    
    # Определяем переменные
    variables = list(expr.free_symbols)  # Автоматически находим все переменные
    
    # Вычисляем градиент
    gradient = [sp.diff(expr, var) for var in variables]
    
    # Вывод результата
    for var, partial_derivative in zip(variables, gradient):
        print(f"∂/∂{var}: {partial_derivative}")

In [14]:
print("Градиент функции Розенброка:")
compute_gradient('100 * (y - x**2)**2 + (x - 1)**2')
print("Градиент функции Била:")
compute_gradient('((1.5 - x + x * y)**2 + (2.25 - x + x * y**2)**2 + (2.625 - x + x * y**3)**2)')

Градиент функции Розенброка:
∂/∂x: -400*x*(-x**2 + y) + 2*x - 2
∂/∂y: -200*x**2 + 200*y
Градиент функции Била:
∂/∂x: 2.25*(1.33333333333333*y - 1.33333333333333)*(0.666666666666667*x*y - 0.666666666666667*x + 1) + 5.0625*(0.888888888888889*y**2 - 0.888888888888889)*(0.444444444444444*x*y**2 - 0.444444444444444*x + 1) + 6.890625*(0.761904761904762*y**3 - 0.761904761904762)*(0.380952380952381*x*y**3 - 0.380952380952381*x + 1)
∂/∂y: 15.75*x*y**2*(0.380952380952381*x*y**3 - 0.380952380952381*x + 1) + 9.0*x*y*(0.444444444444444*x*y**2 - 0.444444444444444*x + 1) + 3.0*x*(0.666666666666667*x*y - 0.666666666666667*x + 1)


Запишем полученные градиенты в функции:

In [21]:
def rosenbrock_grad(point):
    x, y = point[0], point[1]
    return np.array([-400*x*(-x**2 + y) + 2*x - 2, -200*x**2 + 200*y]) 

def beale_grad(point):
    x, y = point[0], point[1]
    return np.array([2.25*(1.33333333333333*y - 1.33333333333333)*(0.666666666666667*x*y - 0.666666666666667*x + 1) + 5.0625*(0.888888888888889*y**2 - 0.888888888888889)*(0.444444444444444*x*y**2 - 0.444444444444444*x + 1) + 6.890625*(0.761904761904762*y**3 - 0.761904761904762)*(0.380952380952381*x*y**3 - 0.380952380952381*x + 1), 15.75*x*y**2*(0.380952380952381*x*y**3 - 0.380952380952381*x + 1) + 9.0*x*y*(0.444444444444444*x*y**2 - 0.444444444444444*x + 1) + 3.0*x*(0.666666666666667*x*y - 0.666666666666667*x + 1)]) 

Теперь подсчитаем минимумы:

In [ ]:
rosenbrock_gradient_descent = GradientDescent()
beale_gradient_descent = GradientDescent()

res = rosenbrock_gradient_descent.minimize(rosenbrock, rosenbrock_grad, (-3, -3))

In [ ]:

import numpy as np
import plotly.graph_objects as go

def rosenbrock_2d(x, y):
    """
    Функция Розенброка для двух переменных.
    """
    return 100 * (y - x**2)**2 + (x - 1)**2





# Пример использования
if __name__ == "__main__":
    # Реальная точка минимума
    x_min_real, y_min_real = 1, 1

    # Пути оптимизации (пример)
    optimization_paths = [
        [(0, 0), (0.5, 0.2), (0.8, 0.5), (1.0, 0.8), (1.0, 1.0)],
        [(0, 0), (0.6, 0.3), (0.9, 0.7), (1.0, 1.0)],
        [(0, 0), (0.4, 0.1), (0.7, 0.4), (0.9, 0.9), (1.0, 1.0)]
    ]




In [4]:
compute_gradient('(1.5-x+xy)^2+(2.25-x+xy^2)^2+(2.625-x+xy^3)^2')

Градиент функции:
∂/∂x: 6.0*x - 2.0*xy**3 - 2.0*xy**2 - 2.0*xy - 12.75
∂/∂xy: -2.0*x + 15.75*xy**2*(-0.380952380952381*x + 0.380952380952381*xy**3 + 1) + 9.0*xy*(-0.444444444444444*x + 0.444444444444444*xy**2 + 1) + 2.0*xy + 3.0
